# Import libraries

In [30]:
import numpy as np
import pandas as pd

In [31]:
import os
import boto3
import redshift_connector

# Extract


In [36]:
df_exito =pd.DataFrame(pd.read_csv("D:/Proyectos/english/exito.csv", encoding="latin-1"))
df_falabella =pd.DataFrame(pd.read_csv("D:/Proyectos/english/falabella.csv", sep=";", encoding="latin-1"))
df_ktronix =pd.DataFrame(pd.read_csv("D:/Proyectos/english/ktronix.csv", sep=";", encoding="latin-1"))


# Transform

In [37]:
df_stores = df_exito.append(df_falabella, ignore_index=True)
df_stores = df_stores.append(df_ktronix, ignore_index=True)
df_stores


C:\Users\danie\AppData\Local\Temp\ipykernel_27796\2657102327.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_stores = df_exito.append(df_falabella, ignore_index=True)
C:\Users\danie\AppData\Local\Temp\ipykernel_27796\2657102327.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_stores = df_stores.append(df_ktronix, ignore_index=True)


,link,brand,warranty,processor,ram,hardDrive,grapCard,capGrapCard,opSystem,screenSize,price
0,https://www.exito.com/computador-portatil-hp-p...,HP,1 año,Intel Core i5-1135G7,8 GB,256 GB SSD,NaN,NaN,Windows 11,15.6 Pulgadas,2.060.859
1,https://www.exito.com/computador-asus-vivobook...,ASUS,1 año,Intel Core i5-12500H,8 GB,512 GB SSD,NaN,NaN,Windows 11,14 Pulgadas,3.199.360
2,https://www.exito.com/portatil-asus-vivobook-1...,ASUS,1 año,AMD Ryzen 5 4500U,8 GB,512 GB SSD,NaN,NaN,Windows 11,14 Pulgadas,1.979.400
3,https://www.exito.com/computador-portatil-hp-a...,HP,1 año,AMD Ryzen 5 5500U,8 GB,512 GB SSD,NaN,NaN,Windows,15.6 Pulgadas,2.089.450
4,https://www.exito.com/computador-portatil-asus...,ASUS,1 año,Intel Core i5-1240P,8 GB,512 GB SSD,NaN,NaN,Windows 11,15.6 Pulgadas,2.759.900
...,...,...,...,...,...,...,...,...,...,...,...
1225,https://www.ktronix.com/macbook-pro16pulgadas-...,APPLE,12 Meses,Apple M2 Pro,16 GB,512 GB SSD,NaN,NaN,MacOS,16 Pulgadas,15.999.000
1226,https://www.ktronix.com/macbook-air-13-pulgada...,APPLE,12 Meses,APPLE M2,8 GB,Estado Solido SSD 512 GB,NaN,NaN,MacOS,13.6 Pulgadas,9.834.000
1227,https://www.ktronix.com/macbook-air-13-pulgada...,APPLE,12 Meses,APPLE M2,8 GB,Estado Solido SSD 512 GB,NaN,NaN,MacOS,13.6 Pulgadas,9.834.000
1228,https://www.ktronix.com/macbook-air-13-pulgada...,APPLE,12 Meses,APPLE M2,8 GB,Estado Solido SSD 512 GB,NaN,NaN,MacOS,13.6 Pulgadas,9.834.000


## Clean Column Ram

In [38]:
df_stores['ram_GB'] = df_stores['ram'].str.replace(' GB', '')
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace('GB', '')
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace('gb', '')
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace('768 MB', np.isnan)
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace('768', np.isnan)
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace('45KB', np.isnan)
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace('1TB', np.isnan)
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace(' MB', '')
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace('MB', '')
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace('No aplica', np.isnan)
df_stores['ram_GB'] = df_stores['ram_GB'].str.replace(' ', '')

## Clean column Hard Drive


In [39]:
df_stores['hardDrive_GB'] = df_stores['hardDrive'].str.replace(' GB', '')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('GB', '')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('gb', '')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('1TB', '1000')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('1tb', '1000')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('1 TB', '1000')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('1 TB', '1000')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('4tb', '4000')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('2TB', '2000')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('Mec', np.isnan)
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('H', np.isnan)
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('Disco s', np.isnan)
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('SSD 960', '960 SSD')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('SSD 480', '480 SSD')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('SSD 240', '240 SSD')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('No aplica', np.isnan)
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('Estado Solido SSD 256', '256 SSD')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('Estado Solido SSD 512', '512 SSD')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('Estado Solido SSD 1000', '1000 SSD')
df_stores['hardDrive_GB'] = df_stores['hardDrive_GB'].str.replace('Estado Solido SSD 2 TB', '2000 SSD')
df_stores['hardDrive_GB'].drop_duplicates()

0         256 SSD
1         512 SSD
5        1000 SSD
13            512
25        128 SSD
45            NaN
63            256
111          1000
174           128
207           500
309       960 SSD
414       240 SSD
449            64
610       480 SSD
614       SSD 100
741           240
770            16
773          2000
870             1
909           250
948             8
1004         4000
1084           32
1139     512  SSD
1183    1000  SSD
1217     2000 SSD
Name: hardDrive_GB, dtype: object

## Clean Price Column

In [ ]:
df_stores['price'] = df_stores['price'].str.replace('.', '')
df_stores = df_stores[df_stores['price'].notnull()]
df_stores['price'] = df_stores['price'].astype('int32')
type(df_stores['price'])

C:\Users\danie\AppData\Local\Temp\ipykernel_27796\1221022348.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_stores['price'] = df_stores['price'].str.replace('.', '')


pandas.core.series.Series

## Clean Warranty Column

In [ ]:
df_stores['warranty'] = df_stores['warranty'].str.replace('1 año', '12')
df_stores['warranty'] = df_stores['warranty'].str.replace(' meses', '')
df_stores['warranty'] = df_stores['warranty'].str.replace('1 Año', '12')
df_stores['warranty'] = df_stores['warranty'].str.replace('2 Años', '24')
df_stores['warranty'] = df_stores['warranty'].str.replace('2 años', '24')
df_stores['warranty'] = df_stores['warranty'].str.replace(' Meses', '')
df_stores['warranty'] = df_stores['warranty'].str.replace(' MESES', '')
df_stores['warranty'] = df_stores['warranty'].str.replace('3 años', '36')
df_stores['warranty'] = df_stores['warranty'].str.replace('1 A', '12')
df_stores['warranty'] = df_stores['warranty'].str.replace('12s', '12')

## Clean OS Column

In [ ]:
df_stores['OS'] = df_stores['opSystem'].str.replace('Windows 11 Home 64 Single Language', 'Windows 11')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 11 Home', 'Windows 11')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 11 Home 64', 'Windows 11')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 11 Home Single Language', 'Windows 11')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 11 home', 'Windows 11')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 11 Single Language', 'Windows 11')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 11 64', 'Windows 11')
df_stores['OS'] = df_stores['OS'].str.replace('Windows  11', 'Windows 11')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 11 Pro', 'Windows 11')

df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 Home', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 Home 64', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('windows 10 profesional no licenciado de prueba', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 de Prueba', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 de prueba', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 de prueba', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 profesional', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 Pro 64', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('windows 10 home', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 home', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 64', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 Profesional', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('windows 10', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 Pro', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 Pro', 'Windows 10')
df_stores['OS'] = df_stores['OS'].str.replace('Windows 10 HOME', 'Windows 10')

df_stores['OS'] = df_stores['OS'].str.replace('windows', 'Windows')
df_stores['OS'] = df_stores['OS'].str.replace('Windows de prueba', 'Windows')


df_stores['OS'] = df_stores['OS'].str.replace('LINUX', 'Linux')
df_stores['OS'] = df_stores['OS'].str.replace('linux', 'Linux')
df_stores['OS'] = df_stores['OS'].str.replace('linux', 'Linux')
df_stores['OS'] = df_stores['OS'].str.replace('linux', 'Linux')
df_stores['OS'] = df_stores['OS'].str.replace('Sistema Operativo LINUX', 'Linux')

df_stores['OS'] = df_stores['OS'].str.replace('Mac Os X', 'Mac OS')
df_stores['OS'] = df_stores['OS'].str.replace('macOS', 'Mac OS')
df_stores['OS'] = df_stores['OS'].str.replace('MacOS', 'Mac OS')
df_stores['OS'] = df_stores['OS'].str.replace('Mac Os', 'Mac OS')


df_stores['OS'] = df_stores['OS'].str.replace('Google Chrome Os', 'Chrome os')

df_stores['OS'] = df_stores['OS'].str.replace('DOS', 'FreeDos')
df_stores['OS'] = df_stores['OS'].str.replace('FreeDOS', 'FreeDos')
df_stores['OS'] = df_stores['OS'].str.replace('FreeFreeDos', 'FreeDos')
df_stores['OS'] = df_stores['OS'].str.replace('Free dos', 'FreeDos')

df_stores['OS'] = df_stores['OS'].str.replace('Sin Sistema Operativo', np.isnan)
df_stores['OS'] = df_stores['OS'].str.replace('No aplica', np.isnan)
df_stores['OS'] = df_stores['OS'].str.replace('De fabricante', np.isnan)

df_stores['OS'] = df_stores['OS'].str.replace('Endless os', 'Endless')
df_stores['OS'] = df_stores['OS'].str.replace('Endless os', 'Endless')


df_stores['OS'].drop_duplicates()

0                   Windows 11
3                      Windows
9                   Windows 10
15                     Endless
48                     FreeDos
64                       Linux
67                      Ubuntu
159                        NaN
168                     Mac OS
400    Sistema Operativo Linux
414                     UBUNTU
487                  Chrome os
797                 Android tv
854              Windows phone
Name: OS, dtype: object

In [ ]:
df_stores.columns

Index(['link', 'brand', 'warranty', 'processor', 'ram', 'hardDrive',
       'grapCard', 'capGrapCard', 'opSystem', 'screenSize', 'price', 'ram_GB',
       'hardDrive_GB', 'OS'],
      dtype='object')

## Create CVS

In [ ]:
df_stores.to_csv('df_stores.csv', index=False)

# Load

In [ ]:
client = boto3.client(
    's3',
    aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
)

conn = redshift_connector.connect(
    host= os.environ.get('host'),
    database=os.environ.get('database'),
    port=5439,
    user=os.environ.get('user'),
    password=os.environ.get('password')
)
cursor = conn.cursor()

In [ ]:
conn.close()

In [ ]:
def load_file(file_name):
    table_name= file_name.split('.')[0]
    client.upload_file(
        Filename='D:/Proyectos/english/{}'.format(file_name),
        Bucket = 'nombre-unico-bucket-platzi',
        Key = 'english_project/{}'.format(file_name)
    )
    sentence = '''copy public/.{} from 's3://nombre-unico-bucket-platzi/{}' credentials 'aws_access_key_id={};aws_secret_access_key={}' csv delimiter '|' region 'us-east-2' ignoreheader 1'''.format(table_name, file_name, os.environ.get('AWS_ACCESS_KEY_ID'),  os.environ.get('AWS_SECRET_ACCESS_KEY'))
    try:
        cursor.execute(sentence)
        print('OK en la tabla ' +table_name)
    except:
        print('Error en la tabla ' +table_name)

In [ ]:
load_file('df_stores.csv')

Error en la tabla df_stores


In [ ]:
conn.commit()

InterfaceError: connection is closed

In [ ]:
conn.close()

InterfaceError: connection is closed